In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
try:
    os.chdir("./drive/My Drive/RuCode/RuCode3/Рекомендации Аниме")
except Exception as e:
    print(e)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip install tensorflow==1.15

In [ ]:
import numpy as np
import tensorflow as tf
from tf_recsys.tfcf.metrics import mae, rmse
from tf_recsys.tfcf.config import Config
from tf_recsys.tfcf.models.svd import SVD
from tf_recsys.tfcf.models.svdpp import SVDPP
from sklearn.model_selection import train_test_split
import pandas as pd
import random
 
np.random.seed(42)
random.seed(42)
tf.set_random_seed(42)
 
ratings = pd.read_csv("user_ratings.csv", dtype=np.int32)
ratings = ratings.loc[ratings["rating"] != 0]
x = ratings[["user_id", "anime_id"]].values
y = ratings["rating"].values
 
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.01, random_state=0)
 
config = Config()
config.num_users = np.max(x[:, 0]) + 1
config.num_items = np.max(x[:, 1]) + 1
config.min_value = np.min(y)
config.max_value = np.max(y)
 
with tf.Session() as sess:
    tf.set_random_seed(42)
    model = SVD(config, sess)
 
    print("Start train")
    model.train(x_train,
                y_train,
                validation_data=(x_test, y_test),
                epochs=15,
                batch_size=1024)
 
    y_pred = model.predict(x_test)
    print('rmse: {}, mae: {}'.format(rmse(y_test, y_pred), mae(y_test, y_pred)))
 
    model.save_model('model/')
 
    sample = pd.read_csv("sample_submission.csv")
    sample_new = np.array([[int(i) for i in id.split()] for id, rat in sample.values])
    test_preds = model.predict(sample_new)
    sample["rating"] = pd.Series([float(x) for x in test_preds])
    sample.to_csv("tfcf_sub30.csv", index=None)

Start train



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Epoch 1/15
10544/10544 [==============================] - 46s 4ms/step - rmse: 1.3603 - mae: 1.0186 - val_rmse: 1.2793 - val_mae: 0.9559
Epoch 2/15
10544/10544 [==============================] - 46s 4ms/step - rmse: 1.2512 - mae: 0.9335 - val_rmse: 1.2560 - val_mae: 0.9373
Epoch 3/15
10544/10544 [==============================] - 46s 4ms/step - rmse: 1.2272 - mae: 0.9143 - val_rmse: 1.2347 - val_mae: 0.9193
Epoch 4/15
10544/10544 [==============================] - 45s 4ms/step - rmse: 1.2056 - mae: 0.8966 - val_rmse: 1.2215 - val_